In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
import cv2
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

In [ ]:
#set up file and df to store accuracy and loss of modelfit epochs
#commented out after creating the df for all 8 optimizers
# file = "modelfit_history.csv"
# modelfit_hist_df = pd.read_csv(file)

In [ ]:
# modelfit_hist_df

In [ ]:
# the kaggle zip file extract put the files in folders for each note type
# to simplify the below code, all files were copied out of their subdirectories into single "Images" folder. 
#Images folder is part of gitignore
counter = 0 #for testing/break purposes only
img_data_array=[]
class_name=[]
folder_name = "Images"
#This code based on https://towardsdatascience.com/loading-custom-image-dataset-for-deep-learning-models-part-1-d64fa7aaeca6
for file in os.listdir(os.path.join(folder_name)):
    counter +=1
    image_path= os.path.join(folder_name,  file)
    image= cv2.imread( image_path, cv2.IMREAD_GRAYSCALE)
    #image=np.array(image)  ????
#     print(image.shape)
    image = image.astype('float32')
    image /= 255 #this gets black and white to 1s and 0s 
    img_data_array.append(image)
    note_class = file[0:1]
#     print(note_class)
    class_name.append(note_class)
#    if (counter>5):
#        break;


In [ ]:
def create_dataset(notes_folder):
    class_name = []
    img_data_array = []
    
    for note_dir in os.listdir(notes_folder):
        
        for file in os.listdir(os.path.join(notes_folder, note_dir)):
            image = os.path.join(notes_folder, note_dir, file)
            image = cv2.imread(notes_folder, cv2.COLOR_BGR2RGB)
            #image = cv2.resize( image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
            #image = np.array(image)  # converts the image to a numpy array
            image = image.astype('float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(note_dir)
            
    print(img_data_array[0])
    return img_data_array, class_name

In [ ]:
# Assigning a unique number to each class
note_classifier_dict = {key:value for value, key in enumerate(np.unique(class_name))}

In [ ]:
note_classifier_dict

In [ ]:
# This is y, converting the class names to a numeric value for all values

target_val=  [note_classifier_dict[class_name[i]] for i in range(len(class_name))]

In [ ]:
# target_val

In [ ]:
# Examine the image array ... setting the threshold to 4096 or above avoids truncating the array

np.set_printoptions(threshold=4096)
img_data_array[0]

In [ ]:
#test to make sure our images are still good.
import matplotlib.pyplot as plt
plt.imshow(img_data_array[10], cmap=plt.cm.Greys)

In [ ]:
# flatten the 64x64 pixels in to a 1D array 
#similar to what was done in MNIST class example
num_dimensions = 64*64

#need to make this an np array so we can use reshape
x=np.array(img_data_array, np.float32)

reshape_test = x.reshape(x.shape[0],num_dimensions)

In [ ]:
reshape_test.shape

In [ ]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(reshape_test,target_val, random_state=42)

## Scaling and Normalization

In [ ]:
##### We performed the scaling manually above using 'image /= 255' #####
# Next, we normalize our training data to be between 0 and 1
# scaler = MinMaxScaler().fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# X_train[0]

## One-Hot Encoding

In [ ]:
# Our Training and Testing labels are integer encoded from 0 to 4
y_train[:20]

In [ ]:

num_classes = 5
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_train[0]

## Building our Model


In [ ]:
# Create an empty sequential model
model = Sequential()

In [ ]:
# Add the first layer where the input dimensions are the 4096 pixel values
# Activation function ... starting with 'relu'
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))


### Add a second hidden layer with 100 densely connected nodes

In [ ]:
model.add(Dense(100, activation='relu'))

### Final output layer uses softmax activation function for logistic regression

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

### Model Summary

In [ ]:
model.summary()

### Compile and Train our Model 

In [ ]:
optimizer = "Adam"
# optimizer = "SGD"
# optimizer = "RMSprop"
# optimizer = "Adadelta"
# optimizer = "Adagrad"
# optimizer = "Adamax"
# optimizer = "Nadam"
# optimizer = tf.keras.optimizers.Ftrl(learning_rate=.001)

In [ ]:
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train (fit) our model using the training data

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=0
    )

In [ ]:
#modelfit_hist_df=pd.DataFrame()   #only needed for first run
# modelfit_hist_df["ftrl_loss"] = history.history['loss']  #only needed for last one
# modelfit_hist_df["ftrl_acc"] = history.history['acc']   #only needed for last one

In [ ]:
# modelfit_hist_df[optimizer+"_loss"] = history.history['loss']
# modelfit_hist_df[optimizer+"_acc"] = history.history['acc']

In [ ]:
# modelfit_hist_df #test output

In [ ]:
# modelfit_hist_df.to_csv(file, index=False)

In [ ]:
#history.model #testing output

In [ ]:
# identify visualizations

In [ ]:
plt.plot(history.history['loss'], label='Loss (training data)')
# plt.plot(history.history['acc'], label='MAE (validation data)')
plt.plot(history.history['acc'], label='Accuracy (training data)')
plt.title('Loss and Accuracy vs. Epochs - Optimizer: '+ optimizer)
plt.ylabel('Model Efficiency')
plt.xlabel('Epochs')
plt.legend(loc="middle")
plt.savefig("Results/100epoch_relu_" + optimizer + ".jpeg")
plt.show()

In [ ]:
# commented out so we don't resave over existing file
#model.save("Results/Notes_relu_" + optimizer + ".h5")

## Evaluating the Model

Evaluate each of the 8 optimizers and store the loss and accuracy in a new DF

In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
#define list to hold model eval results
df_list = []


In [ ]:
def model_predictions(results_folder):

    
     for model_file in os.listdir(os.path.join(results_folder)):
            if "h5" in model_file:
                print("working on file: "+model_file)
                model = load_model(os.path.join(results_folder,model_file))
#                 break #for testing purposes only
                try:
                    model_loss, model_accuracy = model.evaluate(X_test,y_test, verbose=2)
                    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
                    df_list.append([model_file, model_loss,model_accuracy])
                except:
                    print("Ftrl model defies being saved and will not load. filename:"+model_file)
                
                
                

In [ ]:
model_predictions('Results')

In [ ]:
# put results to df
print(df_list)
results_df = pd.DataFrame(df_list,columns=["Model","Loss","Accuracy"])

In [ ]:
#save to csv for later loading/graphing elsewhere
results_df.to_csv(os.path.join('Results','Model_predict_loss_acc.csv'),index=False)